In [1]:
import nltk

nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download()

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\GLC\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GLC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GLC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [75]:
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text.lower())
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Return the filtered tokens as a string
    return ' '.join(filtered_tokens)

# Example usage
text = "This is an example sentence. We're going to tokenize it."
preprocessed_text = preprocess_text(text)
print(preprocessed_text)

example sentence 're going tokenize


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object to extract features
vectorizer = CountVectorizer()
# Fit the vectorizer to the preprocessed text
corpus = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]
vectorizer.fit_transform([preprocess_text(text) for text in corpus])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 540571 stored elements and shape (2000, 39626)>

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import random

# Create a list of (preprocessed text, category) tuples
corpus = [(preprocess_text(movie_reviews.raw(fileid)), category)
          for category in movie_reviews.categories()
          for fileid in movie_reviews.fileids(category)]  # Added the missing parts

# Shuffle the corpus to ensure a random distribution
random.shuffle(corpus)

# Split the corpus into features and labels
texts, labels = zip(*corpus)

# Train a Multinomial Naive Bayes classifier
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
clf = MultinomialNB()
clf.fit(X, labels)


MultinomialNB()

In [90]:
import pickle
f = open('models/car_classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()
f = open('models/car_vectorizer.pickle', 'rb')
vectorizer = pickle.load(f)
f.close()

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import random
# Define a function to generate a chatbot response
def generate_response(vectorizer:CountVectorizer,model:MultinomialNB,user_input):
    # Preprocess and tokenize the user input
    preprocessed_input = preprocess_text(user_input)
    input_vector = vectorizer.transform([preprocessed_input])
    # Use the classifier to predict a response
    predicted_category = model.predict(input_vector)[0]
    # Choose a random movie review from the predicted category
    reviews_in_category = movie_reviews.fileids(predicted_category)
    review_id = random.choice(reviews_in_category)
    review_text = movie_reviews.raw(review_id)
    # Return the review text as the chatbot response
    return review_text

In [77]:
import pandas as pd

In [80]:
import os

In [82]:
os.getcwd()
os.chdir('d:\\MUNTUR AI\\API\\muntur')

In [92]:
corpus = pd.read_excel("training/chat.xlsx")
corpus.head()

,Questions,Answers
0,Pourquoi ma voiture ne démarre pas ?,Les causes possibles incluent une batterie déc...
1,Pourquoi ma voiture fait-elle un bruit étrange...,Les bruits étranges peuvent être causés par de...
2,Pourquoi ma voiture surchauffe-t-elle ?,La surchauffe peut être due à un niveau de liq...
3,Pourquoi le voyant moteur s’allume-t-il sur le...,Le voyant moteur peut s'allumer pour de nombre...
4,Pourquoi ma voiture tire-t-elle d'un côté en c...,Cela peut être causé par un problème de géomét...


In [93]:
corpus['Questions'].to_csv('training/mini/mini_questions.txt', index=False, header=False)
corpus['Answers'].to_csv('training/mini/mini_answers.txt', index=False, header=False)

In [94]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import random

# Shuffle the corpus to ensure a random distribution
# random.shuffle(corpus)

# Split the corpus into features and labels
texts, labels = corpus['Questions'],corpus['Answers']

# Train a Multinomial Naive Bayes classifier
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)
clf = MultinomialNB()
clf.fit(X, labels)

MultinomialNB()

In [95]:
import pickle
f = open('models/car_classifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
import pickle
f = open('models/car_vectorizer.pickle', 'wb')
pickle.dump(vectorizer, f)
f.close()

In [85]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import random
# Define a function to generate a chatbot response
def generate_response_car(vectorizer:CountVectorizer,model:MultinomialNB,user_input):
    # Preprocess and tokenize the user input
    preprocessed_input = preprocess_text(user_input)
    input_vector = vectorizer.transform([preprocessed_input])
    predicted_category = model.predict(input_vector)[0]
    
    return predicted_category

In [86]:
# Define a function to generate a chatbot response
def generate_response_car_preloaded(user_input):
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.feature_extraction.text import CountVectorizer
    import random
    # Preprocess and tokenize the user inputimport pickle
    f = open('models/car_classifier.pickle', 'rb')
    classifier = pickle.load(f)
    f.close()
    f = open('models/car_vectorizer.pickle', 'rb')
    vectorizer = pickle.load(f)
    f.close()
    preprocessed_input = preprocess_text(user_input)
    input_vector = vectorizer.transform([preprocessed_input])
    predicted_category = classifier.predict(input_vector)[0]
    
    return predicted_category

In [88]:
text='pas bon'
response = generate_response_car(vectorizer,clf,text)
print(response)

Les causes possibles incluent une batterie déchargée, un problème de démarreur, un alternateur défectueux ou un problème de carburant.


In [91]:
text='pas bon'
response = generate_response_car_preloaded(text)
print(response)

Les causes possibles incluent une batterie déchargée, un problème de démarreur, un alternateur défectueux ou un problème de carburant.


In [68]:
import os
os.getcwd()

'd:\\MUNTUR AI\\API\\muntur'

In [69]:
os.chdir("d:\\MUNTUR AI\\API\\static")

In [71]:
os.listdir("admin")

['css', 'fonts', 'img', 'js']